In [157]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split


from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report

import glob, os
import numpy as np
from PIL import Image
from skimage.transform import resize
import tensorflow as tf

In [112]:
def Read_Resize_Normalize_images(image_path,L):
    
    if L == 'Au':
        label = 1
    else:
        label = 0
    
    try:
        if image_path != '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Au/Thumbs.db' or image_path != '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/_list.txt' or image_path != '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/Thumbs.db':
            
            img = Image.open(image_path)

            new_shape = (32, 32)

            resized_image = resize(np.array(img), new_shape, anti_aliasing=True)

            # Normalizing
            X = np.array(resized_image, dtype=np.float32).transpose(2, 0, 1) / 255.0
            
            if X.shape == (4,32,32):
                print(image_path)

            tensor_image = torch.from_numpy(X)

            return (tensor_image,label)
    
    except:
        print(image_path)
    

In [113]:
authentic = os.listdir('/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Au')
tampered = os.listdir('/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp')


In [114]:
p_a =  '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Au'
p_t = '/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp'


A = Read_Resize_Normalize_images(p_a+'/'+authentic[1],'Au')
A[0].shape

torch.Size([3, 32, 32])

In [115]:
import os
dataset = []

# Reading Authentic Images
for i in authentic:
    path = os.path.join(p_a+ '/'+ i)
    A = Read_Resize_Normalize_images(path,'Au')
    dataset.append(A)
    
    

# Reading Tampered images
imagesT = []   
for i in tampered:
    path = os.path.join(p_t+ '/'+ i)
    A = Read_Resize_Normalize_images(path,'Tp')
    dataset.append(A)


/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Au/Thumbs.db
/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/_list.txt
/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/Thumbs.db
/kaggle/input/casia-20-image-tampering-detection-dataset/CASIA2/Tp/Tp_D_NRD_S_B_ani20002_nat20042_02437.tif


In [116]:
print(len(dataset))
dataset = list(filter(lambda data: data is not None, dataset))
print(len(dataset))

12617
12614


In [118]:
import random
## Shuffling the list

random.shuffle(dataset)

## Defining My Model

In [289]:
class Image_Manipulation_Model(nn.Module):
    def __init__(self):
        super(Image_Manipulation_Model,self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        
        self.down_conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            self.relu,
            self.maxpool,
        )
        
        self.down_conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            self.relu,
            self.maxpool,
        )
        
        self.down_conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            self.relu,
            self.maxpool,
        )
        
        self.linear = nn.Sequential(
            nn.Linear(in_features=128*4*4, out_features=1024),
            nn.BatchNorm1d(1024),
            self.relu,
            nn.Linear(in_features=1024, out_features=128),
            nn.BatchNorm1d(128),
            self.relu,
            nn.Linear(in_features=128, out_features=2),
            nn.Softmax(dim=1),
        )


    def forward(self, img):
        d1 = self.down_conv1(img)
        d2 = self.down_conv2(d1)
        d3 = self.down_conv3(d2)
        d3 = d3.view(-1, d3.shape[1]*d3.shape[2]*d3.shape[3])
        out = self.linear(d3)
        return out


In [290]:
model = Image_Manipulation_Model()

epochs = 10
batch_size = 128
opt = optim.Adam(model.parameters(), lr=0.0004)
loss_fn1 = nn.CrossEntropyLoss(reduction='mean')

In [291]:
print(model)
print("Parameters: ",sum(p.numel() for p in model.parameters() if p.requires_grad))

Image_Manipulation_Model(
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (down_conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down_conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (down_conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, c

In [292]:
# create a random image tensor of shape (batch_size, 3, 32, 32)
image = torch.randn(128, 3, 32, 32)

# create an instance of the Image_Manipulation_Model class
model = Image_Manipulation_Model()

# pass the image tensor to the model
output = model(image)

# print the output shape
print(output)  # should output torch.Size([1, 2])

tensor([[0.4932, 0.5068],
        [0.6436, 0.3564],
        [0.6093, 0.3907],
        [0.4000, 0.6000],
        [0.5843, 0.4157],
        [0.4083, 0.5917],
        [0.6809, 0.3191],
        [0.5111, 0.4889],
        [0.3607, 0.6393],
        [0.4247, 0.5753],
        [0.6374, 0.3626],
        [0.4680, 0.5320],
        [0.5332, 0.4668],
        [0.3676, 0.6324],
        [0.3652, 0.6348],
        [0.3845, 0.6155],
        [0.4017, 0.5983],
        [0.4914, 0.5086],
        [0.6834, 0.3166],
        [0.4912, 0.5088],
        [0.4017, 0.5983],
        [0.3726, 0.6274],
        [0.5049, 0.4951],
        [0.5857, 0.4143],
        [0.4453, 0.5547],
        [0.4453, 0.5547],
        [0.3658, 0.6342],
        [0.4133, 0.5867],
        [0.5795, 0.4205],
        [0.6031, 0.3969],
        [0.3629, 0.6371],
        [0.5823, 0.4177],
        [0.4237, 0.5763],
        [0.7787, 0.2213],
        [0.3793, 0.6207],
        [0.4541, 0.5459],
        [0.4166, 0.5834],
        [0.4295, 0.5705],
        [0.3

In [294]:
batch_size = 128
train_dataset, validation_dataset, test_dataset = random_split(dataset, [8614, 3000, 1000])
train_data = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
validation_data = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=True)
test_data = DataLoader(dataset=test_dataset, batch_size=128, shuffle=True)

In [295]:
# for x, y in validation_dataset:
#     print(x.shape,y)

In [296]:
for epoch in range(1, epochs+1):
    
    print('Epoch: {}/{} '.format(epoch,epochs))

    model.train()
    train_loss = []

    try:
        for idx, (X, y) in enumerate(train_data):
    #         if X is not None and y is not None or (X,y) is not None:
            X, y = X, y


            opt.zero_grad()
            with torch.set_grad_enabled(True):

                output = model(X)

                loss = loss_fn1(output, y)

                # loss.type(torch.FloatTensor)
                train_loss.append(loss.item())

                loss.backward()
                opt.step()

            print('|--- Train Loss: {}'.format(np.mean(train_loss)))
        
        if(epoch % 2 == 0):
            val_loss = []
            val_acc = []
            with torch.no_grad():

                    model.eval()
                    for id, (X_val, y_val) in enumerate(validation_data):
                        X_val, y_val = X_val, y_val

                        output = model(X_val)
                        loss1 = loss_fn1(output, y_val)
                        val_loss.append(loss1.item())
                        _, y_pred = torch.max(output, dim=1)
                        val_acc.append(accuracy_score(y_val.numpy(), y_pred.numpy()))

                    print('|--- Val Loss: {} --- Val Accuracy: {}'.format(np.mean(val_loss), np.mean(val_acc)))
                # print('F1 score for validation: %f'%(np.mean(epoch_val_score)))

        print('-'*80)
    except:
        continue

Epoch: 1/10 
|--- Train Loss: 0.6684401035308838
|--- Train Loss: 0.6793340146541595
|--- Train Loss: 0.6789886554082235
|--- Train Loss: 0.6842653751373291
|--- Train Loss: 0.6836652040481568
|--- Train Loss: 0.6850891709327698
|--- Train Loss: 0.6868431653295245
|--- Train Loss: 0.68777796626091
|--- Train Loss: 0.6873221529854668
|--- Train Loss: 0.6885446786880494
|--- Train Loss: 0.6895647157322277
|--- Train Loss: 0.6893447786569595
|--- Train Loss: 0.6890011017139142
|--- Train Loss: 0.6890389110360827
|--- Train Loss: 0.6898701985677084
|--- Train Loss: 0.6894473917782307
|--- Train Loss: 0.6904397326357224
|--- Train Loss: 0.6904568043020036
|--- Train Loss: 0.6902438935480619
|--- Train Loss: 0.6899227976799012
|--- Train Loss: 0.6898846683048067
|--- Train Loss: 0.6898377619006417
|--- Train Loss: 0.6896309230638586
|--- Train Loss: 0.6890202139814695
|--- Train Loss: 0.6885170340538025
|--- Train Loss: 0.6881594474499042
|--- Train Loss: 0.6883819081165172
|--- Train Loss: 

In [297]:
ttloss = []
ttacc = []
ttf1 = []
with torch.no_grad():
    model.eval()
    for id, (X_val, y_val) in enumerate(test_data):
        X_val, y_val = X_val, y_val

        output = model(X_val)
        loss1 = loss_fn1(output, y_val)
        ttloss.append(loss1.item())
        _, y_pred = torch.max(output, dim=1)
        y_val, y_pred = y_val.numpy(), y_pred.numpy()
        ttacc.append(accuracy_score(y_val, y_pred))
        ttf1.append(f1_score(y_val, y_pred))
        res = confusion_matrix(y_val, y_pred)
        report = classification_report(y_val, y_pred)

    print('|--- Test Loss: {} --- Test Accuracy: {} --- F1 Score: {}'.format(np.mean(ttloss), np.mean(ttacc), np.mean(ttf1)))
    print('Confusion matrix:\n',res)
    print('Report:\n',report)

|--- Test Loss: 0.6942233219742775 --- Test Accuracy: 0.5297475961538461 --- F1 Score: 0.6127028793693678
Confusion matrix:
 [[19 27]
 [23 35]]
Report:
               precision    recall  f1-score   support

           0       0.45      0.41      0.43        46
           1       0.56      0.60      0.58        58

    accuracy                           0.52       104
   macro avg       0.51      0.51      0.51       104
weighted avg       0.51      0.52      0.52       104



In [235]:
# save model
# torch.save(model,'/kaggle/working/model_IMD.pth')

In [ ]:
# citations 

# https://github.com/Torak28/Image-manipulation-detection
# https://github.com/z1311/Image-Manipulation-Detection
# https://paperswithcode.com/task/image-manipulation-detection

